## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import datetime, os

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
working_df = application_df.drop(['EIN', 'NAME'], axis=1)
working_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   APPLICATION_TYPE        34299 non-null  object
 1   AFFILIATION             34299 non-null  object
 2   CLASSIFICATION          34299 non-null  object
 3   USE_CASE                34299 non-null  object
 4   ORGANIZATION            34299 non-null  object
 5   STATUS                  34299 non-null  int64 
 6   INCOME_AMT              34299 non-null  object
 7   SPECIAL_CONSIDERATIONS  34299 non-null  object
 8   ASK_AMT                 34299 non-null  int64 
 9   IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 2.6+ MB


In [3]:
# Determine the number of unique values in each column.
working_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
working_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = working_df['APPLICATION_TYPE'].value_counts(dropna=False)[working_df['APPLICATION_TYPE'].value_counts(dropna=False)<528]
# application_types_to_replace=['T17', 'T15','T29', 'T14', 'T25', 'T2','T12', 'T13', 'T9']
application_types_to_replace
# Replace in dataframe
for app in application_types_to_replace.index:
    working_df['APPLICATION_TYPE'] = working_df['APPLICATION_TYPE'].replace(app,"Other")

# # Check to make sure binning was successful
working_df['APPLICATION_TYPE'].value_counts()


APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
working_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
working_df['CLASSIFICATION'].value_counts(dropna=False)[working_df['CLASSIFICATION'].value_counts(dropna=False)<1883]

CLASSIFICATION
C7000    777
C1700    287
C4000    194
C5000    116
C1270    114
        ... 
C4120      1
C8210      1
C2561      1
C4500      1
C2150      1
Name: count, Length: 66, dtype: int64

In [8]:
classifications_to_replace = working_df['CLASSIFICATION'].value_counts(dropna=False)[working_df['CLASSIFICATION'].value_counts(dropna=False)<1883]
classifications_to_replace.index

Index(['C7000', 'C1700', 'C4000', 'C5000', 'C1270', 'C2700', 'C2800', 'C7100',
       'C1300', 'C1280', 'C1230', 'C1400', 'C7200', 'C2300', 'C1240', 'C8000',
       'C7120', 'C1500', 'C1800', 'C6000', 'C1250', 'C8200', 'C1238', 'C1278',
       'C1235', 'C1237', 'C7210', 'C2400', 'C1720', 'C4100', 'C1257', 'C1600',
       'C1260', 'C2710', 'C0', 'C3200', 'C1234', 'C1246', 'C1267', 'C1256',
       'C2190', 'C4200', 'C2600', 'C5200', 'C1370', 'C1248', 'C6100', 'C1820',
       'C1900', 'C1236', 'C3700', 'C2570', 'C1580', 'C1245', 'C2500', 'C1570',
       'C1283', 'C2380', 'C1732', 'C1728', 'C2170', 'C4120', 'C8210', 'C2561',
       'C4500', 'C2150'],
      dtype='object', name='CLASSIFICATION')

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

# Replace in dataframe
for cls in classifications_to_replace.index:
    working_df['CLASSIFICATION'] = working_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
working_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies_df=pd.get_dummies(working_df[['ORGANIZATION', 'USE_CASE', 'APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS', 'STATUS', 'ASK_AMT']])
dummies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 43 columns):
 #   Column                        Non-Null Count  Dtype
---  ------                        --------------  -----
 0   STATUS                        34299 non-null  int64
 1   ASK_AMT                       34299 non-null  int64
 2   ORGANIZATION_Association      34299 non-null  bool 
 3   ORGANIZATION_Co-operative     34299 non-null  bool 
 4   ORGANIZATION_Corporation      34299 non-null  bool 
 5   ORGANIZATION_Trust            34299 non-null  bool 
 6   USE_CASE_CommunityServ        34299 non-null  bool 
 7   USE_CASE_Heathcare            34299 non-null  bool 
 8   USE_CASE_Other                34299 non-null  bool 
 9   USE_CASE_Preservation         34299 non-null  bool 
 10  USE_CASE_ProductDev           34299 non-null  bool 
 11  APPLICATION_TYPE_Other        34299 non-null  bool 
 12  APPLICATION_TYPE_T10          34299 non-null  bool 
 13  APPLICATION_TYPE_T19          3

In [11]:
# Split our preprocessed data into our features and target arrays
X=dummies_df.copy()
y=working_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [13]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 43 columns):
 #   Column                        Non-Null Count  Dtype
---  ------                        --------------  -----
 0   STATUS                        34299 non-null  int64
 1   ASK_AMT                       34299 non-null  int64
 2   ORGANIZATION_Association      34299 non-null  bool 
 3   ORGANIZATION_Co-operative     34299 non-null  bool 
 4   ORGANIZATION_Corporation      34299 non-null  bool 
 5   ORGANIZATION_Trust            34299 non-null  bool 
 6   USE_CASE_CommunityServ        34299 non-null  bool 
 7   USE_CASE_Heathcare            34299 non-null  bool 
 8   USE_CASE_Other                34299 non-null  bool 
 9   USE_CASE_Preservation         34299 non-null  bool 
 10  USE_CASE_ProductDev           34299 non-null  bool 
 11  APPLICATION_TYPE_Other        34299 non-null  bool 
 12  APPLICATION_TYPE_T10          34299 non-null  bool 
 13  APPLICATION_TYPE_T19          3

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation='relu', input_dim=43))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=15, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 30)                1320      
                                                                 
 dense_1 (Dense)             (None, 15)                465       
                                                                 
 dense_2 (Dense)             (None, 1)                 16        
                                                                 
Total params: 1801 (7.04 KB)
Trainable params: 1801 (7.04 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
logdir = os.path.join('logs',datetime.datetime.now().strftime("%Y%m%d=%H%M%S"))
weight_callback = tf.keras.callbacks.ModelCheckpoint(logdir, epoch=5, save_weights_only=True, monitor='val_accuracy', save_best_only = False)

In [16]:
# Train the model
nn_fit=nn.fit(X_train_scaled, y_train, epochs=50, callbacks=[weight_callback])

Epoch 1/50
804/804 [==============================] - 4s 3ms/step - loss: 0.5790 - accuracy: 0.7155
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5565 - accuracy: 0.7295
Epoch 3/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5526 - accuracy: 0.7307
Epoch 4/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5510 - accuracy: 0.7321
Epoch 5/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5493 - accuracy: 0.7317
Epoch 6/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5489 - accuracy: 0.7323
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5477 - accuracy: 0.7323
Epoch 8/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5475 - accuracy: 0.7330
Epoch 9/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5465 - accuracy: 0.7321
Epoch 10/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5462 - accuracy: 0.7329

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5508 - accuracy: 0.7290 - 547ms/epoch - 2ms/step
Loss: 0.5507637858390808, Accuracy: 0.7289795875549316


In [18]:
nn.save('AlphabetSoupCharity.h5', overwrite=True, save_format="h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
